In [1]:
import mysql.connector

In [2]:
# CONFIG_USER = "admin"
# CONFIG_PASSWORD = "adminadmin"
# CONFIG_HOST = "sf-fires-1.cmbkjcuzgzbs.us-east-2.rds.amazonaws.com"

CONFIG_USER = "root"
CONFIG_PASSWORD = "AlieN3242DucE-"
CONFIG_HOST = "localhost"

conn = mysql.connector.connect(
    user=CONFIG_USER, 
    password=CONFIG_PASSWORD,
    host=CONFIG_HOST,
    buffered=True
)

cursor = conn.cursor()

In [3]:
DB_NAME = 'sf_fires_testing'

In [4]:
def create_database(cursor):
    try:
        statement = f"CREATE DATABASE {DB_NAME} DEFAULT CHARACTER SET 'utf8'"
        cursor.execute(statement)
    except mysql.connector.Error as err:
        print(f"Failed creating database: {err}")

try:
    cursor.execute(f"USE {DB_NAME}")
    print(f"Using {DB_NAME}")
except mysql.connector.Error as err:
    print(f"Database {DB_NAME} does not exist")
    if err.errno == mysql.connector.errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print(f"Database {DB_NAME} created successfully.")
        conn.database = DB_NAME
    else:
        print(err)

Using sf_fires_testing


In [5]:
query = "show databases"
cursor.execute(query)
for db in cursor:
     print(db[0])

information_schema
mysql
performance_schema
sf_fires_testing
sys


### Create Table

In [6]:
import json

In [14]:
import pickle

# with open('col_data_types.pickle', 'rb') as handle:
#     columns = pickle.load(handle)

with open('rs_data_types.pickle', 'rb') as handle:
    columns = pickle.load(handle)

In [15]:
print(len(columns.keys()))

65


In [16]:
for i, (k, v) in enumerate(columns.items()):
    print(k, v)
    if i > 5: break

incident_number INTEGER
exposure_number SMALLINT
suppression_units SMALLINT
suppression_personnel SMALLINT
ems_units SMALLINT
ems_personnel SMALLINT
other_units SMALLINT


In [22]:
# RDS MySQL
# create_str = "CREATE TABLE IF NOT EXISTS sf_fires ("
# for k, v in columns.items():
#     create_str += f"`{k.lower()}` {v},"
# create_str += "PRIMARY KEY (id)) ENGINE=InnoDB"

# 
create_str = "CREATE TABLE IF NOT EXISTS sf_fires ("
for k, v in columns.items():
    create_str += f"{k.lower()} {v},"
create_str = create_str[:-1] + ")"

In [23]:
create_str

'CREATE TABLE IF NOT EXISTS sf_fires (incident_number INTEGER,exposure_number SMALLINT,suppression_units SMALLINT,suppression_personnel SMALLINT,ems_units SMALLINT,ems_personnel SMALLINT,other_units SMALLINT,other_personnel SMALLINT,estimated_property_loss DECIMAL(7,2),estimated_contents_loss DECIMAL(7,2),fire_fatalities SMALLINT,fire_injuries SMALLINT,civilian_fatalities SMALLINT,civilian_injuries SMALLINT,number_of_alarms SMALLINT,floor_of_fire_origin DECIMAL(7,2),number_of_floors_with_minimum_damage DECIMAL(7,2),number_of_floors_with_significant_damage DECIMAL(7,2),number_of_floors_with_heavy_damage DECIMAL(7,2),number_of_floors_with_extreme_damage DECIMAL(7,2),number_of_sprinkler_heads_operating DECIMAL(7,2),incident_date TIMESTAMP,alarm_dttm TIMESTAMP,arrival_dttm TIMESTAMP,latitude DECIMAL(13,10),longitude DECIMAL(13,10),id VARCHAR,address VARCHAR,call_number VARCHAR,close_dttm VARCHAR,city VARCHAR,zipcode VARCHAR,battalion VARCHAR,station_area VARCHAR,box VARCHAR,first_unit_on_s

In [32]:
drop_table_query = f"DROP TABLE IF EXISTS `sf_fires`"
cursor.execute(drop_table_query)

In [33]:
try:
    cursor.execute(create_str)
    print("table created")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("This table already exists")
    else:
        print(err.msg)

table created


In [34]:
select_query = "SELECT * FROM information_schema.tables"
cursor.execute(select_query)
tables = cursor.fetchall()
print(tables[0])

('def', 'mysql', 'columns_priv', 'BASE TABLE', 'InnoDB', 10, 'Dynamic', 0, 0, 16384, 0, 0, 4194304, None, datetime.datetime(2021, 11, 27, 16, 29, 25), None, None, 'utf8_bin', None, 'row_format=DYNAMIC stats_persistent=0', 'Column privileges')


### Populate Table

In [16]:
import json

In [17]:
with open('Exploration/data.json') as json_file:
    data = json.load(json_file)

In [18]:
data_short = data[:10000]

In [19]:
# building INSERT query
# insert_query = "INSERT INTO sf_fires ("
# for col in columns:
#     insert_query += f"{col}, "
# insert_query = f"{insert_query[:-2]}) "

# row = "(" + ("%s, " * len(columns))[:-2] + "), "
# values_query = "VALUES " + (row * len(data_short))[:-2]
# insert_query = insert_query + values_query
# insert_query[:1500]

# building INSERT query
insert_query = "INSERT INTO sf_fires ("
for col in columns:
    insert_query += f"{col}, "
insert_query = f"{insert_query[:-2]}) VALUES ("

for col in columns:
    insert_query += f"%({col})s, "
insert_query = f"{insert_query[:-2]})"
insert_query

'INSERT INTO sf_fires (incident_number, exposure_number, suppression_units, suppression_personnel, ems_units, ems_personnel, other_units, other_personnel, estimated_property_loss, estimated_contents_loss, fire_fatalities, fire_injuries, civilian_fatalities, civilian_injuries, number_of_alarms, floor_of_fire_origin, number_of_floors_with_minimum_damage, number_of_floors_with_significant_damage, number_of_floors_with_heavy_damage, number_of_floors_with_extreme_damage, number_of_sprinkler_heads_operating, incident_date, alarm_dttm, arrival_dttm, id, address, call_number, close_dttm, city, zipcode, battalion, station_area, box, first_unit_on_scene, primary_situation, mutual_aid, action_taken_primary, action_taken_secondary, action_taken_other, detector_alerted_occupants, property_use, area_of_fire_origin, ignition_cause, ignition_factor_primary, ignition_factor_secondary, heat_source, item_first_ignited, human_factors_associated_with_ignition, structure_type, structure_status, fire_spread,

In [20]:
data_short[0]["point"]["coordinates"]

[-122.41837339, 37.74208979]

In [21]:
for item in data_short:
    insert_query_values = { k:(item[k] if k in item.keys() else None) for k in columns }
    point = item.get("point")
    if point:
        coords = point.get("coordinates")
        insert_query_values["point"] = str(coords)
    else:
        insert_query_values["point"] = None
    cursor.execute(insert_query, insert_query_values)

conn.commit()